In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By # 웹사이트의 구성요소를 선택하기 위해 By 모듈을 불려옵니다
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지가 전부 로드될때까지 기다리는 (Explicitly wait) 기능을 하는 모듈입니다
from webdriver_manager.chrome import ChromeDriverManager # 크롬에서 크롤링을 하기 위해, 웹 드라이버를 설치하는 모듈입니다
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support import expected_conditions as EC # 크롬의 어떤 부분의 상태를 확인하는 모듈입니다
import time # 정해진 시간만큼 기다리게 하기 위한 패키지입니다
from selenium.webdriver.common.keys import Keys

In [2]:
Tearchervile_url= "https://www.teacherville.co.kr/trainguide/review/reviewList.edu"

In [3]:
driver = webdriver.Chrome()
driver.get(Tearchervile_url)

In [4]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="wrap"]'))
    )
finally:
    pass

In [5]:
fnm = '' # 맨 첫번째 상호 이름입니다. 다음 페이지에서도 똑같은 상호가 나온다면, 다음 페이지가 없다고 인식하고, brk 값이 바뀌면서 반복문이 종료되지요
brk = 1
nxt = 0
res = [] # 결과 파일은 판다스 데이터프레임으로 입력할겁니다
page_count = 1


while True:
    
    time.sleep(1)
    hs = driver.page_source
    soup = BeautifulSoup( hs, 'html.parser')
    h = soup.find('ul',{'id':'reviewResultList'})
    
    for i in h.find_all('li'):
        date = i.find('span','day').text
        hour = i.find('strong','credit').text
        name = i.find_all('p')[1].text.split(']')[1].strip()
        cate = i.find_all('p')[1].text.split(']')[0].split('/')[1].strip()
        text = i.find('p','text on').text.strip()
        star = len(i.find_all('span','star on'))
        res.append([hour,name,text,star,cate,date])

    page_count += 1
    brk += 1
    print(page_count, brk)
    
    time.sleep(0.5)
    
    # 다음 페이지로 넘어가는 코드입니다    
    if page_count+nxt != 11 or page_count+nxt != 12:
        driver.find_element(By.XPATH, f'//*[@id="divPaging"]/a[{page_count+nxt}]').send_keys(Keys.ENTER)

    else:
        if brk == 10:
            driver.find_element(By.XPATH, '//*[@id="divPaging"]/a[10]').send_keys(Keys.ENTER)
            page_count = 1
            nxt += 2
   
        else:
            driver.find_element(By.XPATH, '//*[@id="divPaging"]/a[12]').send_keys(Keys.ENTER)
            page_count = 1
            print('tttttt')
    time.sleep(2)

    if brk == 25:
        driver.quit()
        break


            





2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="divPaging"]/a[14]"}
  (Session info: chrome=118.0.5993.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6E9AE8EF2+54786]
	(No symbol) [0x00007FF6E9A55612]
	(No symbol) [0x00007FF6E990A64B]
	(No symbol) [0x00007FF6E994B79C]
	(No symbol) [0x00007FF6E994B91C]
	(No symbol) [0x00007FF6E9986D87]
	(No symbol) [0x00007FF6E996BEAF]
	(No symbol) [0x00007FF6E9984D02]
	(No symbol) [0x00007FF6E996BC43]
	(No symbol) [0x00007FF6E9940941]
	(No symbol) [0x00007FF6E9941B84]
	GetHandleVerifier [0x00007FF6E9E37F52+3524194]
	GetHandleVerifier [0x00007FF6E9E8D800+3874576]
	GetHandleVerifier [0x00007FF6E9E85D7F+3843215]
	GetHandleVerifier [0x00007FF6E9B85086+694166]
	(No symbol) [0x00007FF6E9A60A88]
	(No symbol) [0x00007FF6E9A5CA94]
	(No symbol) [0x00007FF

In [ ]:
df = pd.DataFrame(res)
df.to_csv('./res_Tsherpa.csv', index=False) # 데이터가 실시간으로 저장되도록 합니다
df